In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Big Data Day 2 Afternoon Assignment

In this assignment, we will learn about machine learning with Dask. We will use the market basket dataset loaded below and cluster our data.

In [2]:
# !pip install distributed>=2.4.0 --quiet

# !pip install dask-ml --quiet
# !pip install -U ipykernel --quiet

<IPython.core.display.Javascript object>

In [3]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

<IPython.core.display.Javascript object>

In [4]:
market_basket = dd.read_csv("data/Mall_Customers.csv")

<IPython.core.display.Javascript object>

In [5]:
market_basket.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


<IPython.core.display.Javascript object>

To cluster our data, we must first examine and process it. The first step is to check for missing data. Do this in the cell below. If there is missing data, drop all rows containing missing data.

In [6]:
# Answer below:
market_basket.isna().mean().compute()


CustomerID                0.0
Gender                    0.0
Age                       0.0
Annual Income (k$)        0.0
Spending Score (1-100)    0.0
dtype: float64

<IPython.core.display.Javascript object>

Next, we will get rid of the customer ID column. Drop the column and assign the resulting dataframe to a new variable.

In [7]:
# Answer below:
df = market_basket.drop(['CustomerID'],1)


<IPython.core.display.Javascript object>

In [8]:
df.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


<IPython.core.display.Javascript object>

We will now create a dummy variable from the gender variable.

In [11]:
df["Gender"].value_counts().compute()

Female    112
Male       88
Name: Gender, dtype: int64

<IPython.core.display.Javascript object>

In [ ]:
df["Amount2"] = df["Amount"].apply(lambda x: 1 if x>10000 else 0, meta=('Amount2', 'int64'))

In [12]:
# Answer below:
df['gender_bin'] = df['Gender'].apply(lambda x: 1 if x=='Male' else 0, meta=('gender_bin', 'int64'))


<IPython.core.display.Javascript object>

In [14]:
df["gender_bin"].compute()

0      1
1      1
2      0
3      0
4      0
      ..
195    0
196    0
197    1
198    1
199    1
Name: gender_bin, Length: 200, dtype: int64

<IPython.core.display.Javascript object>

Since the data has different scales, we will scale all columns using min max scaling. We will write our own min max scaling function since the minmax scaler will produce numpy arrays instead of dask arrays. Recall that min max scaling requires finding the min and the max. We subtract the min from each observation and divide by the difference between the max and the min. Complete the function below.

In [15]:
df = df.drop("Gender", 1)

<IPython.core.display.Javascript object>

In [20]:
df["Age"].max().compute()

70

<IPython.core.display.Javascript object>

In [ ]:
scaled = (chunked - min_array) / (max_array - min_array)

In [21]:
# Answer below:


def dask_min_max(x):
    # Input: dask column
    max_col = x.max()
    min_col = x.min()

    scaled = (x - min_col) / (max_col - min_col)
    # Output: dask column that has been min max scaled
    return scaled

<IPython.core.display.Javascript object>

In [23]:
dask_min_max(df["Age"]).compute()

0      0.019231
1      0.057692
2      0.038462
3      0.096154
4      0.250000
         ...   
195    0.326923
196    0.519231
197    0.269231
198    0.269231
199    0.230769
Name: Age, Length: 200, dtype: float64

<IPython.core.display.Javascript object>

Transform all columns using the scaler you wrote above.

In [24]:
# Answer below:
for col in df.columns:
    df[col] = dask_min_max(df[col])


<IPython.core.display.Javascript object>

In [25]:
df.head()

,Age,Annual Income (k$),Spending Score (1-100),gender_bin
0,0.019231,0.000000,0.387755,1.0
1,0.057692,0.000000,0.816327,1.0
2,0.038462,0.008197,0.051020,0.0
3,0.096154,0.008197,0.775510,0.0
4,0.250000,0.016393,0.397959,0.0


<IPython.core.display.Javascript object>

Import the dask clustering function and cluster the data. Use 3 clusters and print the centroids.

In [27]:
from dask_ml.cluster import KMeans

<IPython.core.display.Javascript object>

In [28]:
# Answer below
model = KMeans(3)

model.fit(df)


KMeans(n_clusters=3)

<IPython.core.display.Javascript object>

In [29]:
model.cluster_centers_

array([[0.20074224, 0.36612022, 0.68045113, 0.        ],
       [0.41936189, 0.38710879, 0.48480983, 1.        ],
       [0.57902098, 0.35916542, 0.34471243, 0.        ]])

<IPython.core.display.Javascript object>